**OpenAI 모듈 설치, 불러오기 및 API key 설정하기**

In [1]:
import openai
from openai import OpenAI

client = openai.Client(api_key='your API key')

**1. 대화 하기**

**모델 종류 (맞게 써야됨)**
- gpt-3.5-turbo
- gpt-4
- gpt-4-turbo
...

**대표적인 Parameter**
- **system_content:** system에게 할당 되는 prompt ("너는 helpful assistant AI야")
- **user_content:** 사람이 입력하는 prompt ("오늘 점심 메뉴 추천 해줘")
- **temperature:** 0-2까지의 값으로, 높을수록 답변의 다양성이 올라감
- **max_tokens:** 전체 token 수 제한
- **n:** 같은 질문을 n번 반복

Reference: https://platform.openai.com/docs/api-reference/chat/create

In [4]:
# 대화 함수 정의
def gpt_chat(client, system_content, user_content, temperature=1.0, max_tokens=200, n=3):
    response = client.chat.completions.create(
        #model="gpt-3.5-turbo",
        #model="gpt-4",
        #model="gpt-4-turbo-preview",
        model="gpt-4-0125-preview",     # gpt-4-turbo-preview 최신 버전
        messages=[
            {"role": "system", "content": system_content},
            {"role": "user", "content": user_content}
        ],
        temperature=temperature,
        max_tokens=max_tokens,
        n=n
    )
    return [choice.message.content for choice in response.choices]      #API key 응답구조에서 대답 부분만 빼오는 코드

# Prompt 정의
system_content = """
You are an AI assistant that converts a medical report in English (monolingual) into a medical report in English and Korean (bilingual), using the examples given.
Give me a converted English and Korean mixed medical report (bilingual) without any other preamble text.
When converting, write medical-related words in English, and if not, write them into Korean.
A converted report must have at least half of the total words in English.
"""

user_content = """
<example-monolingual medical report>
The patient has undergone a left lower lobectomy, and there is no indication of a recurrent tumor on the bronchial stump. A wedge resection has been performed on the right upper lobe. The consolidation in the right upper lobe, considered a post-operative change, has slightly decreased compared to the previous assessment. There is no visible pleural effusion on the right side. Additionally, there are no findings suggestive of newly developed metastases in the lung parenchyma. No evidence of metastasis to the liver and adrenal gland is noted in the covered upper abdomen as observed in the scan.
</example-monolingual medical report>

<example-bilingual medical report>
Left lower lobectomy가 되어 있으며 bronchial stump에 recurred tumor를 시사할만한 소견 없음. Right upper lobe에 wedge resection이 되어 있는 상태임. Right upper lobe에 post-OP change로 생각되는 consolidation은 이전보다 약간 감소하였음. Right side에 pleural effusion은 보이지 않는 상태임. 그 외에 lung parenchyma에 새로 생긴 metastasis를 시사할만한 소견은 없음. Scan에서 cover된 상복부에서 liver와 adrenal gland에 metastasis를 시사할만한 소견 없음.
</example-bilingual medical report>

<monolingual medical report>
the heart is again moderate-to-severely enlarged. there is mild widening of the vascular pedicle, suggesting fluid overload, somewhat increased. there is mildly prominent pulmonary vascularity suggesting mild congestion. otherwise, the lungs appear clear. there are no definite pleural effusions or pneumothorax. the lateral view, in particular, is limited by soft tissue attenuation but there are similar small osteophytes and minimal wedging among several lower thoracic vertebral bodies. findings suggesting mild vascular congestion. findings suggesting mild vascular congestion.
</monolingual medical report>
"""

In [5]:
responses = gpt_chat(client, system_content, user_content)
for i, response in enumerate(responses):
    print(f"Response {i+1}: {response}")

Response 1: Heart가 moderate-to-severely enlarged로 다시 나타났음. Vascular pedicle의 mild widening이 있어, fluid overload을 시사하며, 이는 약간 증가하였음을 시사함. Pulmonary vascularity가 mildly prominent하여 mild congestion을 시사함. 그 외에 lungs는 clear하게 보임. Pleural effusion이나 pneumothorax는 확실하지 않음. 특히 lateral view는 soft tissue attenuation에 의해 제한되나, 여러 lower thoracic vertebral bodies 사이에 similar small osteophytes와 minimal wedging이 있음. Mild vascular congestion을 시사하는 소견임. Mild vascular congestion을 시사하는 소견임.
Response 2: Heart이 moderate-to-severely enlarged로 다시 보이고 있음. Vascular pedicle의 mild widening이 있으며, 이는 fluid overload를 암시하며, somewhat increased 상태임. Mildly prominent한 pulmonary vascularity가 mild congestion을 암시함. 그 외에 lungs는 clear하게 보임. Definite한 pleural effusions나 pneumothorax는 없음. 특히 lateral view는 soft tissue attenuation에 의해 제한되지만, similar한 small osteophytes와 several lower thoracic vertebral bodies 사이의 minimal wedging이 있음. Mild vascular congestion을 시사하는 소견 있음. Mild vascular congestion을 시사하는 소견 있음.
Response 3: Heart는 다시

**2. 토큰 수 계산**

ref : https://pkgpl.org/2023/09/14/openai-tiktoken%EC%9D%84-%EC%9D%B4%EC%9A%A9%ED%95%9C-%ED%86%A0%ED%81%B0-%EC%88%98-%EA%B3%84%EC%82%B0%ED%95%98%EA%B8%B0/,  
https://platform.openai.com/tokenizer

In [ ]:
import pandas as pd

import tiktoken

df = pd.read_csv('./df_pretrain_new.csv')

encoder = tiktoken.encoding_for_model("gpt-4-0125-preview")

num = 0
report_flag = ''
report_list = []
report_subject = []
token_num = 0

for item in df.itertuples():    # 중복 없이 subject 별 report 1100개 추출
    report = item[5]
    subject = item[3]
    if report != report_flag:
        num += 1
        report_flag = report
        report_list.append(report)
        report_subject.append(subject)
    if num == 1100:
        break
    
for text in report_list:    # 토큰 수 계산
    token_num += len(encoder.encode(text))
    
"""
model : gpt-4-0125-preview
input : $0.01 / 1K tokens -> $0.00001 / 1 token
output : $0.03 / 1K tokens -> $0.00003 / 1 token
"""
costs = (token_num * 0.00001) + (token_num * 0.00003)
print(f'예상되는 총 비용은 {round(costs, 2)}달러입니다.')

예상되는 총 비용은 3.64달러입니다.
